# Fugitive Emissions

**Table of contents**
- Overview
- Setup
  - Get Token
- Utility Functions
    - Function to call API
    - Input format for API
- Example of Fugitive Emission Query
  - Output Explanation
- Related Links

## Overview


Scope 1 emissions are calculations of direct greenhouse gas (GHG) emissions that can be directly controlled by an organization. The source of the emissions is owned by the organization, for example, fossil fuels.

Use this notebook to learn how to calculate fugitive emissions from leaks of greenhouse gasses (GHG) from refrigeration or air conditioning units.


## Setup


Ensure that you have Python installed in your system. Python 3+ is required.

<b>Note:</b>  To run this notebook seamlessly, you must first configure your credentials for GHG Emission APIs in a file named `secrets.ini` in the following format:

```
[EI]
api.api_key = <Your GHG APIs API key>
api.client_id = <Your GHG APIs client Id>

```

Keep the secrets.ini file at an appropriate relative location of this notebook, for example, as specified in the following config.

```config.read('../../auth/secrets.ini')```

In [1]:
# Install the packages below using pip/pip3 based on your python version.
# pip install pandas,configparser,json,IPython
import pandas as pd
import configparser
import requests
import json
from IPython.display import display as display_summary

### Get Token



Run the following code snippet to generate the Auth Bearer Token by using your api_key configured in secrets.ini.

In [13]:
config = configparser.RawConfigParser()
config.read(['../../auth/secrets.ini','../../auth/config.ini'])

EI_API_KEY  = config.get('EI', 'api.api_key')
EI_CLIENT_ID = config.get('EI', 'api.client_id')
EI_AUTH_ENPOINT = config.get('EI', 'api.auth_endpoint')
EI_BASE_URL = config.get('EI', 'api.base_url')

def get_bearer_token(token):
    headers = {
        'Content-Type': 'application/json',
        'cache-control': 'no-cache',
    }
    data = {"apiKey":token, "clientId":EI_CLIENT_ID}
    bearer_token_req = requests.post(EI_AUTH_ENPOINT, headers=headers, data= json.dumps(data))
    if bearer_token_req.status_code != 200:
        print( "Error in getting Bearer token. Error code : ",bearer_token_req.status_code)
    return bearer_token_req.json()['access_token']

mybearer_token = get_bearer_token(EI_API_KEY)


The following code snippet sets the fugitive emission parameters.


## Utility Functions

### Function to call API

In [14]:
def Call_Carbon_API(CO2API, Co2Data, token):
    headers = {
        'accept': 'application/json',
        'Authorization': 'Bearer '+token,
        'Content-Type': 'application/json',
    }

    CO2response_req = requests.post( CO2API, headers=headers, data=json.dumps(Co2Data))
    if CO2response_req.status_code != 200:
        print( "Problem! Error in generating response. Error code : ",CO2response_req.status_code)
    return CO2response_req.json()

### Input format for API

In [18]:
def customer_id(customerid):
    Input =  {
              "id": customerid
          }
    return Input 

def customer_onBehalfOfClient(companyId,companyName):
    Input =  {
              "companyId": companyId,
              "companyName": companyName
          }
    return Input  


def customer_organisation(departmentId,departmentName):
    Input =  {
              "departmentId": departmentId,
              "departmentName": departmentName
          }
    return Input  


def customer_requestType(requestType):
    Input =  {
              "requestType": requestType,
             }
    return Input     


def customer_location(country,stateProvince,zipPostCode,city):
    Input =  {
              "country": country,
              "stateProvince": stateProvince,
              "zipPostCode": zipPostCode,
              "city": city
          }
    return Input        


def customer_site(siteId,siteName,buildingId,buildingName):
    Input = {
              "siteId": siteId,
              "siteName": siteName,
              "buildingId": buildingId,
              "buildingName": buildingName
          }               
    return Input

def fugitive_input(refrigerantName,refrigerantInventoryBeginning,refrigerantInventoryEnd,refrigerantsPurchasedFromProducers,
                   refrigerantsProvidedByManufacturers,refrigerantsAddedToEquipment,
                   refrigerantReturnedAfterRecycling,refrigerantSales,refrigerantLeftInEquipment,refrigerantReturnedToSuppliers,
                   refrigerantForRecycling,refrigerantForDestruction,totalChargeNewEquipment,totalChargeRetrofitted,originalChargeEquipment,
                   totalChargeEquipmentRetroAway,unitOfMeasurement):
    input = {"refrigerantName": refrigerantName,
             "refrigerantInventoryBeginning": refrigerantInventoryBeginning,
             "refrigerantInventoryEnd": refrigerantInventoryEnd,
             "refrigerantsPurchasedFromProducers": refrigerantsPurchasedFromProducers,
             "refrigerantsProvidedByManufacturers": refrigerantsProvidedByManufacturers,
             "refrigerantsAddedToEquipment": refrigerantsAddedToEquipment,
             "refrigerantReturnedAfterRecycling": refrigerantReturnedAfterRecycling,
             "refrigerantSales": refrigerantSales,
             "refrigerantLeftInEquipment": refrigerantLeftInEquipment,
             "refrigerantReturnedToSuppliers": refrigerantReturnedToSuppliers,
             "refrigerantForRecycling": refrigerantForRecycling,
             "refrigerantForDestruction": refrigerantForDestruction,
             "totalChargeNewEquipment": totalChargeNewEquipment,
             "totalChargeRetrofitted": totalChargeRetrofitted,
             "originalChargeEquipment": originalChargeEquipment,
             "totalChargeEquipmentRetroAway": totalChargeEquipmentRetroAway,
             "unitOfMeasurement": unitOfMeasurement
            }
    return input

## Example of Fugitive Emission Query



You can check the accepted values for the individual fields of the request payload. For more information, see [GHG Emissions API Developer Guide](https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/GHG+Emissions+API#fugitive).

Run the following snippet to set the request payload for the Fugitive Emission Endpoint Query:

In [16]:
request_payload ={'customID'        : customer_id(customerid='Retailer_A_fugitive_31234'), 
                   'onBehalfOfClient': customer_onBehalfOfClient(companyId='877889877',
                                                                 companyName='Retailer_A') , 
                   'organisation'    : customer_organisation(departmentId='384834',
                                                             departmentName='Acme_retail'), 
                   'requestType'     : customer_requestType(requestType='ACTUAL'), 
                   'location'        : customer_location(country='England',
                                                         stateProvince='',
                                                         zipPostCode='',
                                                         city='London'),
                   'site'            : customer_site(siteId='T2383823',
                                                     siteName='Retailed_A_South_Bank',
                                                     buildingId='B38383',
                                                     buildingName='Building_6'), 
                   'timePeriod'      : {'year' : '2021', 
                                        'month': '1'},
                   'activityData' : fugitive_input(
                          refrigerantName = 'R-404A',
                          refrigerantInventoryBeginning ='10',
                          refrigerantInventoryEnd ='2',
                          refrigerantsPurchasedFromProducers='6',
                          refrigerantsProvidedByManufacturers ='2',
                          refrigerantsAddedToEquipment ='3',
                          refrigerantReturnedAfterRecycling ='0',
                          refrigerantSales ='3',
                          refrigerantLeftInEquipment ='4',
                          refrigerantReturnedToSuppliers ='1',
                          refrigerantForRecycling ='0',
                          refrigerantForDestruction ='0',
                          totalChargeNewEquipment ='0',
                          totalChargeRetrofitted ='0',
                          originalChargeEquipment ='0',
                          totalChargeEquipmentRetroAway='0',
                          unitOfMeasurement='kilogram')
                   }        
       
print("Requests payload is :",json.dumps(request_payload, indent=2))   


Requests payload is : {
  "customID": {
    "id": "Retailer_A_fugitive_31234"
  },
  "onBehalfOfClient": {
    "companyId": "877889877",
    "companyName": "Retailer_A"
  },
  "organisation": {
    "departmentId": "384834",
    "departmentName": "Acme_retail"
  },
  "requestType": {
    "requestType": "ACTUAL"
  },
  "location": {
    "country": "England",
    "stateProvince": "",
    "zipPostCode": "",
    "city": "London"
  },
  "site": {
    "siteId": "T2383823",
    "siteName": "Retailed_A_South_Bank",
    "buildingId": "B38383",
    "buildingName": "Building_6"
  },
  "timePeriod": {
    "year": "2021",
    "month": "1"
  },
  "activityData": {
    "refrigerantName": "R-404A",
    "refrigerantInventoryBeginning": "10",
    "refrigerantInventoryEnd": "2",
    "refrigerantsPurchasedFromProducers": "6",
    "refrigerantsProvidedByManufacturers": "2",
    "refrigerantsAddedToEquipment": "3",
    "refrigerantReturnedAfterRecycling": "0",
    "refrigerantSales": "3",
    "refrigerantLef

Use the following code to run the Fugitive API:



In [17]:
EI_API_ENDPOINT = f"{EI_BASE_URL}/v2/carbon/fugitive"
CO2Response = Call_Carbon_API(EI_API_ENDPOINT , request_payload, mybearer_token)

df_res = pd.json_normalize(CO2Response)
df_styled = df_res.style.set_table_styles([{
    'selector': 'table',
    'props' : [
        ('display', 'block'),
        ('overflow-x', 'auto'),
        ('white-space', 'nowrap')
    ]
}])
display_summary(df_styled)

,CO2,N2O,CH4,HFC,PFC,SF6,NF3,CO2e,unitOfMeasurement,description
0,0.000000,0.000000,0.000000,43.370800,0.000000,0.000000,0.000000,43.373000,metric tonne,"Sales-Based Method is used. GWP of the blend is calculated using a weighted average from the blend composition and the 100-year GWPs from IPCC Fifth Assessment Report, 2014 (AR5). The blend compositions are from EPA."


### Output Explanation


<b>CO2</b> -              Release of carbon dioxide in metric tonnes

<b>N2O </b>-              Release of nitrous oxide in metric tonnes

<b>CH4 </b>-               Release of methane in metric tonnes

<b>HFC </b>-               Release of hydrofluorocarbon in metric tonnes

<b>PFC </b>-               Release of perfluorocarbon in metric tonnes

<b>SF6 </b>-               Release of sulphur hexafluoride in metric tonnes

<b>NF3 </b>-               Release of nitrogen trifluoride in metric tonnes

<b>CO2e </b> -             Total greenhouse gases emitted in metric tonnes

<b>unitOfMeasurement</b>-  Unit used to measure

## Related Links

[GHG Emissions API Developer Guide](https://developer.ibm.com/apis/catalog/envintelsuite--ibm-environmental-intelligence/GHG+Emissions+API)